# 布林带策略

In [11]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略主体

买入信号：收盘价向上突破布林带上界或下界时买入。

买入仓位：逢信号就买入，每次买入1手。

卖出信号：收盘价向下突破布林带上界或下界时卖出。

卖出仓位：清仓。

### 个股版本

In [6]:
class Bollinger_s(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('b_per', 20),
        ('b_dev', 2),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        # Initialization
        self.order = None
        self.buyprice = None
        self.sellprice = None

        # Alias
        self.dataclose = self.data.close

        # Indicators
        ## Bollinger
        self.bollinger = btind.BollingerBands(self.data.close,
                                              period=self.p.b_per, 
                                              devfactor=self.p.b_dev)

        # Signals
        self.buy_s = bt.Or(bt.And(self.data.close(0) > self.bollinger.top(0), 
                                  self.data.close(-1) < self.bollinger.top(-1)),
                           bt.And(self.data.close(0) > self.bollinger.bot(0), 
                                  self.data.close(-1) < self.bollinger.bot(-1)))
        
        self.sell_s = bt.Or(bt.And(self.data.close(0) < self.bollinger.top(0), 
                                   self.data.close(-1) > self.bollinger.top(-1)),
                            bt.And(self.data.close(0) < self.bollinger.bot(0), 
                                   self.data.close(-1) > self.bollinger.bot(-1)))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price, -order.executed.size,
                         self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        if self.order:
            return
        
        if self.buy_s[0]:
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % 
                     (self.dataclose[0], 100, self.getposition(self.data).size))
            self.order = self.buy(size=100)

        elif self.sell_s[0] and self.getposition(self.data).size > 0:
            self.log('SELL CREATE (Close), Price: %.2f, Lots: %i' % 
                        (self.dataclose[0], self.getposition(self.data).size))
            self.order = self.close()

### 个股版本（允许做多）

In [ ]:
class Bollinger_ss(bt.Strategy):
    params = (
        ('printlog', False),
        ('b_per', 20),
        ('b_dev', 2),
        ('pos_max', 10),
    )
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
        
    
    def __init__(self):

        # Initialization
        self.order = None
        self.buyprice = None
        self.sellprice = None

        # Alias
        self.dataclose = self.data.close

        # Indicators
        self.Bollinger = btind.BollingerBands(self.dataclose, 
                                              period=self.p.b_per, 
                                              devfactor=self.p.b_dev)

        # Signals
        self.buy_s = bt.Or(bt.And(self.dataclose(0) > self.lines.top(0), 
                                  self.dataclose(-1) < self.lines.top(-1)),
                           bt.And(self.dataclose(0) > self.lines.bot(0), 
                                  self.dataclose(-1) < self.lines.bot(-1)))
        
        self.sell_s = bt.Or(bt.And(self.dataclose(0) < self.lines.top(0), 
                                  self.dataclose(-1) > self.lines.top(-1)),
                           bt.And(self.dataclose(0) < self.lines.bot(0), 
                                  self.dataclose(-1) > self.lines.bot(-1)))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Position:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.getposition(self.data).size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Position:%i, Cash: %i, Value: %i' %
                         (order.executed.price, -order.executed.size,
                          self.getposition(self.data).size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))


    def next(self):
        if self.order:
            return
        
        # Check if position size exceeds the limit
        if abs(self.getposition(self.data).size) > self.params.pos_max:
            # Close short position
            if self.getposition(self.data).size < 0:
                self.log('BUY CREATE (Close Short), Price: %.2f' % self.dataclose[0])
                self.order = self.buy(size=-self.getposition(self.data).size)
            # Close long position
            if self.getposition(self.data).size > 0:
                self.log('SELL CREATE (Close Long), Price: %.2f' % self.dataclose[0])
                self.order = self.close()

        lot_ava = (self.broker.get_cash()/2)//(self.data.close[0])

        if self.buy_s[0]:
            if lot_ava > 0:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy(size=lot_ava)
        else:
            if  self.sell_s[0]:
                if lot_ava > 0:
                    self.log('SELL CREATE, %.2f' % self.dataclose[0])
                    self.order = self.sell(size=lot_ava)
                elif lot_ava < 0:
                    self.log('SELL CREATE, %.2f' % self.dataclose[0])
                    self.order = self.sell(size=self.getposition(self.data).size)

    def stop(self):
        self.log('(Period %2d, DevFactor %.2f). Ending Position %i. Ending Value %.2f' %
                 (self.params.b_per, self.params.b_dev, self.getposition(self.data).size, self.broker.getvalue()), doprint = False)

## 回测

### 个股回测

In [7]:
stock_code = '002057.SZ'
startdate = dt.datetime.now() - dt.timedelta(days = 365)
enddate = dt.datetime.now()


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(Bollinger_s, printlog=True) 

    # Create stock Data Feed
    df = utilsJ.get_stock(stock_code, startdate, enddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)
    
    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(10000)

    # Set leverage
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2021-11-29: BUY CREATE, Price: 9.33, Lots: 100, Current Position: 0
2021-11-30: BUY EXECUTED, Price: 9.33, Lot:100, Cash: 9066, Value: 9968
2021-11-30: SELL CREATE (Close), Price: 9.02, Lots: 100
2021-12-01: SELL EXECUTED, Price: 9.02, Lot:100, Cash: 9968, Value: 9968
2021-12-01: OPERATION PROFIT, GROSS -31.53, NET -31.53
2022-01-28: BUY CREATE, Price: 7.89, Lots: 100, Current Position: 0
2022-02-07: BUY EXECUTED, Price: 7.89, Lot:100, Cash: 9179, Value: 9980
2022-02-28: BUY CREATE, Price: 9.30, Lots: 100, Current Position: 100
2022-03-01: BUY EXECUTED, Price: 9.30, Lot:100, Cash: 8248, Value: 10085
2022-03-01: SELL CREATE (Close), Price: 9.19, Lots: 200
2022-03-02: SELL EXECUTED, Price: 9.19, Lot:200, Cash: 10085, Value: 10085
2022-03-02: OPERATION PROFIT, GROSS 117.28, NET 117.28
2022-03-16: BUY CREATE, Price: 7.89, Lots: 100, Current Position: 0
2022-03-17: BUY EXECUTED, Price: 7.89, Lot:100, Cash: 9296, Value: 10124
2022-04-25: SELL CREATE (Close)

### 股指期货回测 (沪深当月)

In [ ]:
future_code = 'IFL.CFX'
startdate = dt.datetime.now() - dt.timedelta(days=365)
enddate = dt.datetime.now()

if __name__ ==  '__main__':
    
    df = utilsJ.future_ts(future_code, startdate, enddate)

    max_np = 0.0
    max_idx = (0, 0.0)
    
    for i in range(3, 31, 1):
        for j in np.arange(0.2, 2.1, 0.1):
            # Create a cerebro entity
            cerebro = bt.Cerebro()

            # Add a strategy
            strats = cerebro.addstrategy(Bollinger_ss, printlog=False, b_per=i, b_dev=float(j)) 

            # Create a Data Feed
            data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)

            # Add the Data Feed to Cerebo
            cerebro.adddata(data)

            # Set our desired cash start
            cerebro.broker = bt.brokers.BackBroker(coc=True)
            
            cerebro.broker.setcash(100000.00)
            
            # Record the starting conditions
            start_value = cerebro.broker.getvalue()
            #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

            # Run over everything
            cerebro.run()

            # Record the final result
            final_value = cerebro.broker.getvalue()
            netp = (final_value / start_value -1)*100
            if netp > max_np:
                max_np = netp
                max_idx = (i, j)
    print(max_np, max_idx)